In [39]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pyreadstat as pyr
import json
import numpy as np
from spss_import import read_sav 
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [40]:
spssfile = f"files/SPSS_Example2.sav"
df, df_meta = read_sav(spssfile)
df.head()

,RID,MARST,PWT,testvar
0,10000001,3,537,1
1,10000002,1,231,1
2,10000003,2,599,1
3,10000004,1,4003,7
4,10000005,4,598,8


In [41]:
spssfile2 = f"files/ESS-Data-Wizard-subset-2023-10-23.dta"
df2, df2_meta = read_sav(spssfile2)
df2.head()

,name,essround,edition,proddate,idno,cntry,dweight,pspwght,pweight,anweight,prob,stratum,psu,nwspol,netusoft,netustm,ppltrst,pplfair,pplhlp
0,ESS9e03_1,9,3.1,17.02.2021,27,AT,0.581174,0.218111,0.302091,0.06589,0.001176,59,1688,60,5,180,2,2,2
1,ESS9e03_1,9,3.1,17.02.2021,137,AT,1.062772,0.413473,0.302091,0.124907,0.000643,79,88,10,5,20,7,8,7
2,ESS9e03_1,9,3.1,17.02.2021,194,AT,1.376509,2.270293,0.302091,0.685836,0.000496,11,938,60,4,180,5,7,7
3,ESS9e03_1,9,3.1,17.02.2021,208,AT,0.993399,0.386483,0.302091,0.116753,0.000688,74,1998,45,5,120,3,9,5
4,ESS9e03_1,9,3.1,17.02.2021,220,AT,0.377353,1.032102,0.302091,0.311789,0.001811,99,601,30,1,a,5,8,4


In [42]:
import pandas as pd

def create_variable_view(df_meta):
    # Extract the attributes from df_meta
    label = df_meta.column_names_to_labels
    values = df_meta.variable_value_labels
    missing = df_meta.missing_ranges
    format = df_meta.original_variable_types
    measure = df_meta.variable_measure

    # Convert dictionaries into individual dataframes
    df_label = pd.DataFrame(list(label.items()), columns=['name', 'label'])
    df_format = pd.DataFrame(list(format.items()), columns=['name', 'format'])
    df_measure = pd.DataFrame(list(measure.items()), columns=['name', 'measure'])

    # For values and missing, handle them differently due to dictionaries/lists inside
    df_values_list = [{'name': k, 'values': str(v)} for k, v in values.items()]  # Convert values to string
    df_values = pd.DataFrame(df_values_list)

    df_missing_list = [{'name': k, 'missing': str(v)} for k, v in missing.items()]  # Convert missing values to string
    df_missing = pd.DataFrame(df_missing_list)

    # Merge dataframes on the 'name' column
    variable_view = df_label
    if not df_values.empty:
        variable_view = variable_view.merge(df_values, on='name', how='outer')
    
    if not df_missing.empty:
        variable_view = variable_view.merge(df_missing, on='name', how='outer')

    variable_view = variable_view \
        .merge(df_format, on='name', how='outer') \
        .merge(df_measure, on='name', how='outer')

    # Ensure 'values' and 'missing' columns are present
    if 'values' not in variable_view.columns:
        variable_view['values'] = pd.NA

    if 'missing' not in variable_view.columns:
        variable_view['missing'] = pd.NA

    return variable_view[['name', 'format', 'measure', 'label', 'values', 'missing']]


create_variable_view(df_meta)

,name,format,measure,label,values,missing
0,RID,F8.0,scale,Respondent ID,NaN,NaN
1,MARST,F8.0,nominal,Marital Status,"{1.0: 'Single', 2.0: 'Married', 3.0: 'In a civ...","[{'lo': 7.0, 'hi': 8.0}, {'lo': 9.0, 'hi': 9.0}]"
2,PWT,F8.2,scale,Person Weight,NaN,"[{'lo': -8.0, 'hi': -8.0}, {'lo': -9.0, 'hi': ..."
3,testvar,A1,nominal,testvar label,"{'1': 'substantive', '7': 'missing1', '8': 'mi...","[{'lo': '7', 'hi': '7'}, {'lo': '8', 'hi': '8'..."


In [43]:
import pandas as pd

def create_variable_view2(df_meta):
    # Extract the attributes from df_meta
    label = df_meta.column_names_to_labels
    values = df_meta.variable_value_labels

    # Convert user-defined missing values to the desired format
    missing = {}
    for key, vals in df_meta.missing_user_values.items():
        missing[key] = [{"lo": val, "hi": val} for val in vals]

    format = df_meta.original_variable_types
    measure = df_meta.variable_measure

    # Convert dictionaries into individual dataframes
    df_label = pd.DataFrame(list(label.items()), columns=['name', 'label'])
    df_format = pd.DataFrame(list(format.items()), columns=['name', 'format'])
    df_measure = pd.DataFrame(list(measure.items()), columns=['name', 'measure'])

    # For values and missing, handle them differently due to dictionaries/lists inside
    df_values_list = [{'name': k, 'values': str(v)} for k, v in values.items()]  # Convert values to string
    df_values = pd.DataFrame(df_values_list)

    df_missing_list = [{'name': k, 'missing': str(v)} for k, v in missing.items()]  # Convert missing values to string
    df_missing = pd.DataFrame(df_missing_list)

    # Merge dataframes on the 'name' column
    variable_view = df_label
    if not df_values.empty:
        variable_view = variable_view.merge(df_values, on='name', how='outer')
    else:
        variable_view['values'] = pd.NA
    
    if not df_missing.empty:
        variable_view = variable_view.merge(df_missing, on='name', how='outer')
    else:
        variable_view['missing'] = pd.NA

    variable_view = variable_view \
        .merge(df_format, on='name', how='outer') \
        .merge(df_measure, on='name', how='outer')

    # Ensure 'values' and 'missing' columns are present
    if 'values' not in variable_view.columns:
        variable_view['values'] = pd.NA

    if 'missing' not in variable_view.columns:
        variable_view['missing'] = pd.NA

    return variable_view

create_variable_view2(df2_meta)

,name,label,values,missing,format,measure
0,name,Title of dataset,NaN,NaN,%9s,unknown
1,essround,ESS round,NaN,NaN,%3.0g,unknown
2,edition,Edition,NaN,NaN,%3s,unknown
3,proddate,Production date,NaN,NaN,%10s,unknown
4,idno,Respondent's identification number,NaN,NaN,%8.0g,unknown
5,cntry,Country,NaN,NaN,%2s,unknown
6,dweight,Design weight,NaN,NaN,%3.2g,unknown
7,pspwght,Post-stratification weight including design we...,NaN,NaN,%3.2g,unknown
8,pweight,Population size weight (must be combined with ...,NaN,NaN,%3.2g,unknown
9,anweight,Analysis weight,NaN,NaN,%4.2g,unknown


In [44]:
Label = df_meta.column_names_to_labels
Values = df_meta.variable_value_labels
Missing = df_meta.missing_ranges
Format = df_meta.original_variable_types
Measure = df_meta.variable_measure

In [45]:
def generate_InstanceVariable(df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#{variable}",
            "@type": "InstanceVariable",
            "name": variable,
            "displayLabel": df_meta.column_labels[idx],
            "hasIntendedDataType": df_meta.original_variable_types[variable]
        }

        # Check if variable has substantive concepts
        if variable in df_meta.variable_value_labels:
            elements['takesSubstantiveConceptsFrom'] = f"#substantiveConceptualDomain-{variable}"

        # Check if variable has sentinel concepts
        if variable in df_meta.missing_ranges or (len(df_meta.missing_ranges) == 0 and variable in df_meta.missing_user_values):
            elements['takesSentinelConceptsFrom'] = f"#sentinelConceptualDomain-{variable}"

        json_ld_data.append(elements)

    return json_ld_data

In [46]:
# MeasureComponent
def generate_MeasureComponent(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names[1:]):
        elements = {
            "@id": f"#measureComponent-{variable}",
            "@type": "MeasureComponent",
            "isDefinedBy": f"#{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [47]:
# MeasureComponent2
def generate_MeasureComponent2(df_meta, varlist=None):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names):
        if variable not in varlist:
            elements = {
                "@id": f"#measureComponent-{variable}",
                "@type": "MeasureComponent",
                "isDefinedBy": f"#{variable}"
            }
            json_ld_data.append(elements)

    return json_ld_data

In [48]:
# IdentifierComponent
def generate_IdentifierComponent(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#identifierComponent-{df_meta.column_names[0]}",
        "@type": "IdentifierComponent",
        "isDefinedBy": f"#{df_meta.column_names[0]}"
    }
    json_ld_data.append(elements)

    return json_ld_data

In [49]:
# IdentifierComponent2
def generate_IdentifierComponent2(df_meta, varlist=None):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names):
        if variable in varlist:
            elements = {
                "@id": f"#identifierComponent-{variable}",
                "@type": "IdentifierComponent",
                "isDefinedBy": f"#{variable}"
            }
            json_ld_data.append(elements)

    return json_ld_data

In [50]:
# logicalRecord
def generate_LogicalRecord(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#logicalRecord",
        "@type": "LogicalRecord",
        "organizes": f"#wideDataSet"
    }
    has = []    
    for x, variable in enumerate(df_meta.column_names):
        has.append(f"#{variable}")
    elements['has'] = has
    json_ld_data.append(elements)
    return json_ld_data

In [51]:
# PhysicalDataset
def generate_PhysicalDataset(df_meta, spssfile):
    json_ld_data = []
    elements = {
        "@id": f"#physicalDataset",
        "@type": "PhysicalDataset",
        "formats": "#dataStore",
        "physicalFileName": spssfile
    }
    has = ["#physicalRecordSegment"]
    elements['has'] = has    
    json_ld_data.append(elements)
    return json_ld_data

In [52]:
# DataStore
def generate_DataStore(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#dataStore",
        "@type": "DataStore",
        "recordCount": df_meta.number_rows
    }
    has = ["#logicalRecord"]
    elements['has'] = has   
    
    json_ld_data.append(elements)
    return json_ld_data

In [53]:
# WideDataSet
def generate_WideDataSet(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataSet",
        "@type": "WideDataSet",
        "isStructuredBy": "#wideDataStructure"
    }
    
    json_ld_data.append(elements)
    return json_ld_data

In [54]:
# WideDataStructure
def generate_WideDataStructure(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataStructure",
        "@type": "WideDataStructure",
    }
    has = ["#primaryKey", f"#identifierComponent-{df_meta.column_names[0]}"]
    
    for x, variable in enumerate(df_meta.column_names[1:]):
        has.append(f"#measureComponent-{variable}")
    elements['has'] = has
        
    json_ld_data.append(elements)
    return json_ld_data

In [55]:
# WideDataStructure2
def generate_WideDataStructure2(df_meta, varlist=None):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataStructure",
        "@type": "WideDataStructure",
    }
    has = ["#primaryKey"]
    
    for x, variable in enumerate(df_meta.column_names):
        if variable in varlist:
            has.append(f"#identifierComponent-{variable}")
        else:
            has.append(f"#measureComponent-{variable}")
    elements['has'] = has
        
    json_ld_data.append(elements)
    return json_ld_data

In [56]:
# PrimaryKey
def generate_PrimaryKey(df_meta):
    json_ld_data = []
    elements = {
        "@id": "#primaryKey",
        "@type": "PrimaryKey",
        "isComposedOf": "#primaryKeyComponent"
    }
        
    json_ld_data.append(elements)
    return json_ld_data

In [57]:
# PrimaryKeyComponent
def generate_PrimaryKeyComponent(df_meta):
    json_ld_data = []
    elements = {
        "@id": "#primaryKeyComponent",
        "@type": "PrimaryKeyComponent",
        "correspondsTo": f"#identifierComponent-{df_meta.column_names[0]}"
    }
        
    json_ld_data.append(elements)
    return json_ld_data


In [58]:
# PrimaryKeyComponent2
def generate_PrimaryKeyComponent2(df_meta, varlist=None):
    json_ld_data = []
    elements = {
        "@id": "#primaryKeyComponent",
        "@type": "PrimaryKeyComponent",
    }
    has = []    
    for variable in varlist:
        has.append(f"#identifierComponent-{variable}")
    
    elements['correspondsTo'] = has      
    json_ld_data.append(elements)
    return json_ld_data

In [59]:
# PhysicalRecordSegment
def generate_PhysicalRecordSegment(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#physicalRecordSegment",
        "@type": "PhysicalRecordSegment",
        "mapsTo": "#logicalRecord",
    }
    has = ["#physicalSegmentLayout"]
    elements['has'] = has    
    
    json_ld_data.append(elements)
    return json_ld_data

In [60]:
# PhysicalSegmentLayout
def generate_PhysicalSegmentLayout(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#physicalSegmentLayout",
        "@type": "PhysicalSegmentLayout",
        "formats": "#logicalRecord",
        "isDelimited": "false",
        "delimiter": "",
    }
    has = []    
    for x, variable in enumerate(df_meta.column_names):
        has.append(f"#valueMapping-{variable}")
        has.append(f"#valueMappingPosition-{variable}")
    elements['has'] = has    
    json_ld_data.append(elements)
    return json_ld_data

In [61]:
# ValueMapping
def generate_ValueMapping(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#valueMapping-{variable}",
            "@type": "ValueMapping",
        }
        datapoints = []
        for i, x in enumerate(df[variable]):
            datapoints.append(f"#{variable}-dataPoint-{i}")
        elements['formats'] = datapoints

        json_ld_data.append(elements)

    return json_ld_data

In [62]:
# ValueMappingPosition
def generate_ValueMappingPosition(df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#valueMappingPosition-{variable}",
            "@type": "ValueMappingPosition",
            "value": idx,
            "indexes": f"#valueMapping-{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [63]:
# DataPoint
def generate_DataPoint(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-dataPoint-{idx}",
                "@type": "DataPoint",
                "isDescribedBy": f"#{variable}-instanceValue-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [64]:
# DataPointPosition
def generate_DataPointPosition(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-dataPointPosition-{idx}",
                "@type": "DataPointPosition",
                "value": idx,
                "indexes": f"#{variable}-dataPoint-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [65]:
# SubstantiveValueDomain
def generate_SubstantiveValueDomain(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels):
        elements = {
            "@id": f"#substantiveValueDomain-{variable}",
            "@type": "SubstantiveValueDomain",
            "takesValuesFrom": f"#codelist.{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [66]:
# InstanceValue
def generate_InstanceValue(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-instanceValue-{idx}",
                "@type": "InstanceValue",
                "content": value,
                "isStoredIn": f"#{variable}-dataPoint-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [67]:
# Codelist
def generate_Codelist(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels.items()):
        elements = {
            "@id": f"#codelist-{variable[x]}",
            "@type": "Codelist",
        }
        has = []
        your_dict = variable[1]
        # Loop through the dictionary and extract the keys
        for key in your_dict.keys():
            codes = {
                "@id": f"#code-{key}-{variable[x]}"
            }
            has.append(codes)
        elements['has'] = has    

        json_ld_data.append(elements)

    return json_ld_data

In [68]:
# Code
def generate_Code(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels.items()):
        your_dict = variable[1]
        # Loop through the dictionary and extract the keys
        for key, value in your_dict.items():
            elements = {
                "@id": f"#code-{key}-{variable[x]}",
                "@type": "Code",
            }
            has = []
            codes = {
                "@id": f"#{key}"
            }
            has.append(codes)
            elements['denotes'] = has

            has = []
            codes = {
                "@id": f"#{value}"
            }
            has.append(codes)
            elements['uses'] = has 

            json_ld_data.append(elements)

    return json_ld_data

In [69]:
# SubstantiveConceptualDomain
def generate_SubstantiveConceptualDomain(df_meta):
    json_ld_data = []
    for variable in (df_meta.variable_value_labels):
        elements = {
            "@id": f"#substantiveConceptualDomain-{variable}",
            "@type": "SubstantiveConceptualDomain",
            "takesConceptsFrom": f"#substantiveConceptScheme-{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [94]:
# SubstantiveConceptScheme
def generate_SubstantiveConceptScheme(df_meta):
    json_ld_data = []

    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        elements = {
            "@id": f"#substantiveConceptScheme-{variable_name}",
            "@type": "skos:ConceptScheme",
        }

        excluded_values = set()

        # Check if variable_name is in relevant_variables
        if variable_name in relevant_variables:
            
            # If the relevant variable data is based on ranges and contains dictionaries
            if isinstance(relevant_variables[variable_name], list) and all(isinstance(item, dict) for item in relevant_variables[variable_name]):
                for dict_range in relevant_variables[variable_name]:
                    lo_is_numeric = isinstance(dict_range['lo'], (int, float)) or (
                        isinstance(dict_range['lo'], str) and dict_range['lo'].isnumeric()
                    )
                    hi_is_numeric = isinstance(dict_range['hi'], (int, float)) or (
                        isinstance(dict_range['hi'], str) and dict_range['hi'].isnumeric()
                    )

                    if lo_is_numeric and hi_is_numeric:
                        excluded_values.update(
                            range(int(float(dict_range['lo'])), int(float(dict_range['hi'])) + 1)
                        )
                    elif isinstance(dict_range['lo'], str):
                        excluded_values.add(dict_range['lo'])
                    else:
                        print(f"Warning: Unsupported 'lo' value: {dict_range['lo']}")

            # If the relevant variable data contains strings (user-defined missing values)
            elif isinstance(relevant_variables[variable_name], list):
                excluded_values.update(set(map(str, relevant_variables[variable_name])))

        # Use list comprehension to generate the hasTopConcept list
        has_top_concept = [
            f"#{variable_name}-concept-{value}"
            for value in values_dict.keys()
            if str(value) not in excluded_values
        ]

        # Only add to json_ld_data if has_top_concept list is not empty
        if has_top_concept:
            elements['skos:hasTopConcept'] = has_top_concept
            json_ld_data.append(elements)

    return json_ld_data

In [71]:
# ValueAndConceptDescription
def generate_ValueAndConceptDescription(df_meta):
    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values
    
    return [
        {
            "@id": f"#valueAndConceptDescription-{variable}",
            "@type": "ValueAndConceptDescription",
            "isDescribedBy": str(value),
        }
        for variable, value in relevant_variables.items()
    ]

In [72]:
# SentinelConceptualDomain
def generate_SentinelConceptualDomain(df_meta):
    json_ld_data = []

    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    for variable in relevant_variables:
        elements = {
            "@id": f"#sentinelConceptualDomain-{variable}",
            "@type": "SentinelConceptualDomain",
            "isDescribedBy": f"#valueAndConceptDescription-{variable}",
        }
        if variable in df_meta.variable_value_labels.keys():
            elements["takesConceptsFrom"] = f"#sentinelConceptScheme-{variable}"
            
        json_ld_data.append(elements)

    return json_ld_data
generate_SentinelConceptualDomain(df_meta)

[{'@id': '#sentinelConceptualDomain-MARST',
  '@type': 'SentinelConceptualDomain',
  'isDescribedBy': '#valueAndConceptDescription-MARST',
  'takesConceptsFrom': '#sentinelConceptScheme-MARST'},
 {'@id': '#sentinelConceptualDomain-PWT',
  '@type': 'SentinelConceptualDomain',
  'isDescribedBy': '#valueAndConceptDescription-PWT'},
 {'@id': '#sentinelConceptualDomain-testvar',
  '@type': 'SentinelConceptualDomain',
  'isDescribedBy': '#valueAndConceptDescription-testvar',
  'takesConceptsFrom': '#sentinelConceptScheme-testvar'}]

In [73]:
df_meta.missing_ranges

{'MARST': [{'lo': 7.0, 'hi': 8.0}, {'lo': 9.0, 'hi': 9.0}],
 'PWT': [{'lo': -8.0, 'hi': -8.0}, {'lo': -9.0, 'hi': -9.0}],
 'testvar': [{'lo': '7', 'hi': '7'},
  {'lo': '8', 'hi': '8'},
  {'lo': '9', 'hi': '9'}]}

In [74]:
df2_meta.missing_user_values

{'netustm': ['a', 'b', 'c', 'd'],
 'pplfair': ['a', 'b', 'c'],
 'nwspol': ['a', 'b', 'c'],
 'pplhlp': ['a', 'b', 'c'],
 'ppltrst': ['a', 'b', 'c'],
 'netusoft': ['a', 'b', 'c']}

In [81]:
# generate_SentinelConceptScheme
def generate_SentinelConceptScheme(df_meta):
    json_ld_data = []

    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    def is_value_in_range(value, ranges):
        """Check if a value is in any of the given ranges."""
        for range_dict in ranges:
            if range_dict['lo'] <= value <= range_dict['hi']:
                return True
        return False

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        elements = {
            "@id": f"#sentinelConceptScheme-{variable_name}",
            "@type": "skos:ConceptScheme",
        }

        has_top_concept = []
        
        if variable_name in relevant_variables:
            if variable_name in df_meta.missing_ranges:
                # Use list comprehension to generate the hasTopConcept list
                has_top_concept = [
                    f"#{variable_name}-concept-{value}"
                    for value in values_dict.keys()
                    if is_value_in_range(value, df_meta.missing_ranges[variable_name])
                ]
            else:
                excluded_values = set(df_meta.missing_user_values[variable_name])
                has_top_concept = [
                    f"#{variable_name}-concept-{value}"
                    for value in values_dict.keys()
                    if value in excluded_values
                ]
        
        # Add the hasTopConcept list to elements
        elements['skos:hasTopConcept'] = has_top_concept
        
        json_ld_data.append(elements)

    return json_ld_data

In [82]:
# Concept
def generate_Concept(df_meta):
    def is_value_in_excluded_ranges(value, excluded_ranges):
        if isinstance(value, (int, float)):
            return value in excluded_ranges
        elif isinstance(value, str):
            return value in excluded_ranges
        return False

    json_ld_data = []

    # Convert user-defined missing values to the desired format
    if df_meta.missing_user_values:
        missing = {}
        for key, vals in df_meta.missing_user_values.items():
            missing[key] = [{"lo": val, "hi": val} for val in vals]
    else:
        missing = df_meta.missing_ranges

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        # Check if variable_name is in missing and, if so, generate the excluded_ranges
        excluded_ranges = set()
        if variable_name in missing:
            for dict_range in missing[variable_name]:
                lo_is_numeric = isinstance(dict_range['lo'], (int, float)) or (
                    isinstance(dict_range['lo'], str) and dict_range['lo'].isnumeric()
                )
                hi_is_numeric = isinstance(dict_range['hi'], (int, float)) or (
                    isinstance(dict_range['hi'], str) and dict_range['hi'].isnumeric()
                )

                if lo_is_numeric and hi_is_numeric:
                    # Case: 'lo' and 'hi' can be converted to int
                    excluded_ranges.update(
                        range(int(float(dict_range['lo'])), int(float(dict_range['hi'])) + 1)
                    )
                elif isinstance(dict_range['lo'], str) and dict_range['lo'] == dict_range['hi']:
                    # Case: 'lo' and 'hi' are the same non-numeric string
                    excluded_ranges.add(dict_range['lo'])
                else:
                    print(f"Warning: Unsupported 'lo' value: {dict_range['lo']}")

        # Iterate through values_dict and create elements, taking into account excluded_keys
        for key, value in values_dict.items():
            elements = {
                "@id": f"#{variable_name}-concept-{key}",
                "@type": "skos:Concept",
                "notation": key,
                "prefLabel": f"{value}",
            }

            # Add the inScheme key to elements based on whether the key is in excluded_ranges
            if is_value_in_excluded_ranges(key, excluded_ranges):
                elements['inScheme'] = f"#sentinelConceptScheme-{variable_name}"
            else:
                elements['inScheme'] = f"#substantiveConceptScheme-{variable_name}"

            # Append elements to json_ld_data inside the loop
            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# Generate JSON-LD
InstanceVariable = generate_InstanceVariable(df_meta)
#SubstantiveValueDomain = generate_SubstantiveValueDomain(df_meta)
SubstantiveConceptualDomain = generate_SubstantiveConceptualDomain(df_meta)
SentinelConceptualDomain = generate_SentinelConceptualDomain(df_meta)
ValueAndConceptDescription = generate_ValueAndConceptDescription(df_meta)
SubstantiveConceptScheme = generate_SubstantiveConceptScheme(df_meta)
SentinelConceptScheme = generate_SentinelConceptScheme(df_meta)
Concept = generate_Concept(df_meta)
#Codelist = generate_Codelist(df_meta)
#Code = generate_Code(df_meta)
LogicalRecord = generate_LogicalRecord(df_meta)
PhysicalDataset = generate_PhysicalDataset(df_meta, spssfile)
PhysicalRecordSegment = generate_PhysicalRecordSegment(df_meta)
PhysicalSegmentLayout = generate_PhysicalSegmentLayout(df_meta)
ValueMapping = generate_ValueMapping(df, df_meta)
ValueMappingPosition = generate_ValueMappingPosition(df_meta)
InstanceValue = generate_InstanceValue(df, df_meta)
DataPoint = generate_DataPoint(df, df_meta)
DataPointPosition = generate_DataPointPosition(df, df_meta)
DataStore = generate_DataStore(df_meta)
WideDataSet = generate_WideDataSet(df_meta)
WideDataStructure = generate_WideDataStructure(df_meta)
PrimaryKey = generate_PrimaryKey(df_meta)
PrimaryKeyComponent = generate_PrimaryKeyComponent(df_meta)
MeasureComponent = generate_MeasureComponent(df_meta)
IdentifierComponent = generate_IdentifierComponent(df_meta)


json_ld_graph = DataStore + PhysicalDataset + PhysicalRecordSegment + PhysicalSegmentLayout + ValueMapping + \
ValueMappingPosition + DataPoint + DataPointPosition + InstanceValue + LogicalRecord + WideDataSet + \
WideDataStructure + IdentifierComponent + MeasureComponent + PrimaryKey + PrimaryKeyComponent + InstanceVariable + \
SubstantiveConceptualDomain + SubstantiveConceptScheme + SentinelConceptualDomain + ValueAndConceptDescription + \
SentinelConceptScheme + Concept

## Create Json

In [ ]:
# Create a dictionary with the specified "@context" and "@graph" keys
json_ld_dict = {
    "@context": [
        "https://ddi-alliance.bitbucket.io/DDI-CDI/DDI-CDI_v1.0-rc1/encoding/json-ld/ddi-cdi.jsonld",
        {
            "skos": "http://www.w3.org/2004/02/skos/core#"
        }
    ],
    "@graph": json_ld_graph
}

def default_encode(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    elif pd.isna(obj):  # Checks for pd.NA
        return None
    elif isinstance(obj, pd.Timestamp):  # Checks for Timestamp
        return obj.isoformat()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")



# Convert the Python dictionary to a JSON string with pretty formatting

json_ld_string = json.dumps(json_ld_dict, indent=4, default=default_encode)
# New code to write the JSON-LD string to a file
with open('output.jsonld', 'w') as file:
    file.write(json_ld_string)

##### 